In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

In [4]:
from june import World 
from june.demography.geography import Geography
from june.demography import Demography
from june.interaction import ContactAveraging
from june.infection import Infection
from june.infection.symptoms import SymptomsConstant
from june.infection.transmission import TransmissionConstant
from june.groups import Hospitals, Schools, Companies, Households, CareHomes, Cemeteries
from june.groups.leisure import Cinemas, Pubs, Groceries
from june.simulator import Simulator
from june.seed import Seed
from june.policy import Policy, Policies
from june import paths
from june.logger.read_logger import ReadLogger
from june.infection.infection import InfectionSelector
from june.world import generate_world_from_hdf5

No --data argument given - defaulting to:
/home/florpi/JUNE/data
No --configs argument given - defaulting to:
/home/florpi/JUNE/configs


# Initialize world

To initialize a certain world, we need to add the different components we want to have in it. First we specify what super areas (msoa) we want to create. We have included these ones, because they are known to contain hospitals, schools, care homes, and companies.

After creating the geography, we create the different components the worlds need to have such as care homes, companies ...

In [6]:
%%time
geography = Geography.from_file({
                                "super_area":  ["E02002512", "E02001697"]
                                          #["E02001720",
                                          #"E00088544", 
                                          #"E02002560", 
                                          #"E02002559"]
                                }
                                )

geography.hospitals = Hospitals.for_geography(geography)
geography.schools = Schools.for_geography(geography)
geography.companies = Companies.for_geography(geography)
geography.care_homes = CareHomes.for_geography(geography)
demography = Demography.for_geography(geography)
world = World(geography, demography, include_households=True)

2020-06-09 10:01:23,692 - june.demography.geography - INFO - There are 43 areas and 2 super_areas in the world.
2020-06-09 10:01:23,701 - june.groups.hospital - INFO - There are 1 hospitals in this geography.
2020-06-09 10:01:23,750 - june.groups.school - INFO - There are 5 schools in this geography.
2020-06-09 10:01:23,766 - june.groups.school - INFO - No school for the age 0 in this world.
2020-06-09 10:01:23,772 - june.groups.school - INFO - No school for the age 1 in this world.
2020-06-09 10:01:23,777 - june.groups.school - INFO - No school for the age 2 in this world.
2020-06-09 10:01:23,825 - june.groups.school - INFO - No school for the age 19 in this world.
2020-06-09 10:01:24,037 - june.groups.carehome - INFO - There are 43 care_homes in this geography.
populating the world's geography with the specified demography...
2020-06-09 10:01:30,618 - numexpr.utils - INFO - NumExpr defaulting to 4 threads.
2020-06-09 10:01:32,441 - june.distributors.worker_distributor - INFO - There 

### If it took a long time to run the previous command, it might be a good idea to save the world to reuse it later.

In [7]:
#world.to_hdf5("world.hdf5")

If we would like to load the world we saved, we just do

In [8]:
#world = generate_world_from_hdf5("world.hdf5")

you have now a beautiful pre-pandemic world. 

## Commute, travel and leisure

We can add cinemas, pubs, groceries, etc. as well as commute in major cities with

In [9]:
# leisure
world.cinemas = Cinemas.for_geography(geography)
world.pubs = Pubs.for_geography(geography)
world.groceries = Groceries.for_super_areas(world.super_areas,
                                            venues_per_capita=1/500)

In [10]:
# commute
world.initialise_commuting()

We are also going to need some cemeteries...geography.cemeteries = Cemeteries()


In [11]:
world.cemeteries = Cemeteries()

# Adding the infection

Now, you can play around with different models of infections. The building blocks for an infection are: Transmission and Symptoms. For now, only SymptomsConstant and TransmissionConstant are in a state to be used.

In [12]:
selector = InfectionSelector.from_file()

In [13]:
selector

# Adding the interaction

In [14]:
interaction = ContactAveraging.from_file(selector=selector)
#interaction = DefaultInteraction.from_file(selector=selector)

Beta are the intensities of the interaction taking place at the different groups

In [15]:
interaction.beta

{'box': 1,
 'pub': 0.1,
 'grocery': 0.1,
 'cinema': 0.1,
 'commute_unit': 0.5,
 'commute_city_unit': 0.5,
 'hospital': 0.5,
 'care_home': 0.5,
 'company': 0.5,
 'school': 0.5,
 'household': 0.5}

to modify these, simply do

In [16]:
interaction.beta['household'] *= 2

In [17]:
interaction.beta

{'box': 1,
 'pub': 0.1,
 'grocery': 0.1,
 'cinema': 0.1,
 'commute_unit': 0.5,
 'commute_city_unit': 0.5,
 'hospital': 0.5,
 'care_home': 0.5,
 'company': 0.5,
 'school': 0.5,
 'household': 1.0}

moreover this interaction module uses contact matrices, that are different for different groups. These contact matrices shouldnt be modified for now. However they are a combination of conversational contact matrices, and physical contact matrices (see the BBC pandemic paper, from where these matrices are extracted https://www.medrxiv.org/content/10.1101/2020.02.16.20023754v2)

There is a parameter, ``alpha`` ($\alpha$), that combines these two matrices in the following way,


$\beta M \left(1 + (\alpha -1) \right) P$

where $\beta$ is the intensity of the interaction, and $P$ the physical contact matrix. A larger $\alpha$ produces more physical contacts. It is an overall number, non dependent of the particular group.


In [18]:
interaction.alpha_physical

2.0

In [19]:
interaction.alpha_physical /= 2

In [20]:
interaction.alpha_physical

1.0

# Seed the disease

There are two options implemented in the seed at the moment, either you specify the number of cases and these are then homogeneously distributed by population to the different areas, or you use UK data on cases per region. For now use the first case.

In [21]:
seed = Seed(world.super_areas, selector,)

In [22]:
n_cases = 50
seed.unleash_virus(n_cases) # play around with the initial number of cases

# Set policies

In [23]:
social_distance = Policy(policy="social_distance", start_time=datetime(2020, 3, 16), end_time=datetime(2020, 3, 20))

In [24]:
policies = Policies.from_file([social_distance])

In [25]:
policies.config['social distancing']

{'alpha factor': 2, 'beta factor': 2}

# Run the simulation

Since the timer configuration is a bit cumbersome, it is read from the config file at ``configs/config_example.yaml``

In [34]:
CONFIG_PATH = "../configs/config_example.yaml"

simulator = Simulator.from_file(
     world, interaction, selector, 
    config_filename = CONFIG_PATH,
    policies = policies
)

In [35]:
simulator.timer.reset()

In [36]:
%%time
simulator.run()

2020-06-09 10:02:22,973 - june.simulator - INFO - Starting group_dynamics for 70 days at day 0
2020-06-09 10:02:22,974 - june.simulator - INFO - starting the loop ..., at 0 days, to run for 70 days
2020-06-09 10:02:23,052 - june.simulator - INFO - Date = 2020-03-07 12:00:00, number of deaths =  0, number of infected = 601
2020-06-09 10:02:23,369 - june.simulator - INFO - Date = 2020-03-08 00:00:00, number of deaths =  0, number of infected = 630
2020-06-09 10:02:23,495 - june.simulator - INFO - Date = 2020-03-08 12:00:00, number of deaths =  0, number of infected = 636
2020-06-09 10:02:23,656 - june.simulator - INFO - Date = 2020-03-09 00:00:00, number of deaths =  0, number of infected = 652
2020-06-09 10:02:23,874 - june.simulator - INFO - Date = 2020-03-09 10:00:00, number of deaths =  0, number of infected = 1027
2020-06-09 10:02:24,042 - june.simulator - INFO - Date = 2020-03-09 12:00:00, number of deaths =  0, number of infected = 1030
2020-06-09 10:02:24,213 - june.simulator - I

AttributeError: 'Policies' object has no attribute 'config_file'

While the simulation runs (and afterwards) we can launch the visualization webpage by running
```python june/visualizer.py path/to/results``` 

# Getting the results

All results are stored in a json file specified in the ``save_path`` variable in the config file. We can also access it from ``world.logger`` directly.

In [29]:
import pandas as pd

In [30]:
read = ReadLogger()

FileNotFoundError: [Errno 2] File /home/florpi/JUNE/data/covid_real_data/n_deaths_region.csv does not exist: '/home/florpi/JUNE/data/covid_real_data/n_deaths_region.csv'

## Hospital data and how it changed over time

In [ ]:
hospitals_df = read.load_hospital_capacity()

In [ ]:
hospitals_df.head(3)

In [ ]:
hospitals_characteristics_df = read.load_hospital_characteristics()

In [ ]:
hospitals_characteristics_df

## where did infections happen?

In [ ]:
loc_df = read.get_locations_infections()

In [ ]:
import matplotlib.ticker as mtick

ax = loc_df['percentage_infections'].sort_values().plot.bar()
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel('Percentage of infections at location')
plt.xlabel('location')

## rate of infection

In [ ]:
read.infections_df

In [ ]:
r_df = read.get_r()

In [ ]:
r_df

In [ ]:
r_df.plot()
plt.axhline(y=1, linestyle='dashed', color='gray')
plt.xlabel('Date')
plt.ylabel('R')

## World infection curves, and by super area

In [ ]:
world_df = read.world_summary()

In [ ]:
world_df.plot()
plt.legend()

In [ ]:
area_df = read.super_area_summary()

In [ ]:
area_df[area_df.super_area == 'E02002560'].plot()
plt.legend()

In [ ]:
area_df[area_df.super_area == 'E02001720'].plot()
plt.legend()

## World infection curves per age group

In [ ]:
ages_df = read.age_summary([0,10,20,30,40,
                  50,60,70,80,90,100])

In [ ]:
for name, group in ages_df.groupby('age_range'):
    group['infected'].plot(label=name)
plt.legend()

## Draw some of the symptoms trajectories

In [ ]:
random_trajectories = read.draw_symptom_trajectories(window_length=100,
                                        n_people=5)

In [ ]:
from june.infection import SymptomTag

In [ ]:
symptoms_values = [tag.value for tag in SymptomTag]
symptoms_names = [tag.name for tag in SymptomTag]


In [ ]:
for df_person in random_trajectories:
    df_person['symptoms'].plot()
plt.ylabel('Symptoms Trajectory')
_ = plt.yticks(symptoms_values, symptoms_names)
plt.xlabel('Date')

In [ ]:
for df_person in random_trajectories:
    df_person['n_secondary_infections'].plot()
plt.ylabel('Number of secondary infections')

plt.xlabel('Date')